## AI Agent Core

**Goal:** Build intelligent agents that generate, plan, and optimize content using foundation models (like GPT, open-source LLMs, etc.) through the **LangChain framework.**

Build **modular, agentic components** that:
| Sub-Agent           | What It Does                                           |
| ------------------- | ------------------------------------------------------ |
| Topic Generator  | Suggests new content topics based on trends + clusters |
| Brief Writer     | Creates outlines and summaries                         |
| SEO Optimizer    | Enhances content for search discoverability            |
| Calendar Planner | Generates a suggested content schedule                 |

agents are **composable** — they can call tools, access memory, use documents, or interact with APIs.

#### Concepts

| Concept            | Explanation                                                                            |
| ------------------ | -------------------------------------------------------------------------------------- |
| **LangChain**      | Python framework to build LLM-powered agents that can use tools, memory, and workflows |
| **PromptTemplate** | Structure + variables used to generate consistent outputs from LLMs                    |
| **Tools**          | Functions that your AI agent can call (e.g., search, summarize, recommend)             |
| **Chains**         | Sequences of steps the LLM performs (e.g., extract → summarize → plan)                 |
| **Memory**         | Lets agents remember past interactions                                                 |
| **Agents**         | Smart, autonomous systems that use all the above to complete tasks                     |


## Step 4.1 - AI Agent Core: Topic Generator

In [1]:
import os
import sys
import pandas as pd

In [2]:
# Add project root to sys path
"""
This is needed because this notebook file is under root/notebooks folder. 
Where as the agent files are under root/src/agents folder.
This is to navigate from notebooks to agents folder
"""

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#print(project_root)
#print(sys.path)

if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
# Now we can import from src
from src.agents.topic_generator import generate_topics

In [4]:
# Load combined data
data_path = os.path.join(project_root, "data", "combined_data.csv")
#print(data_path)

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at: {data_path}")

combined_df = pd.read_csv(data_path, parse_dates=["publishedAt"])
print(f"Loaded data from: {data_path}")
print(f"Total articles/ posts: {len(combined_df)}")

combined_df.head()

Loaded data from: /Users/vinothhaldorai/Documents/Vinoth/PROJECTS/content-marketing-agent/data/combined_data.csv
Total articles/ posts: 46


,title,publishedAt,source,publishedDate
0,AI in content marketing: How creators and mark...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
1,AI in Marketing recent news | Content Marketin...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
2,A Complete Guide to Adopting AI in Content Mar...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
3,AI tools for content marketing : r/marketing,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
4,Artificial Intelligence And The Future Of Cont...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08


#### Generate topics using Agent

In [12]:
# Generate topics using agent
# We will pass recent titles from multiple platforms to LLM

titles = combined_df["title"].dropna().tolist()
# print(titles)

# Call the LLM agent
topics = generate_topics(titles)

# Display suggested topics
print("\nSuggested Topics:\n")
"""
for i, topic in enumerate(topics, 1):
    print(f"{i}. {topic}")
"""
print(topics)


Suggested Topics:

Based on the provided keywords, here are five article topics that are fresh, relevant, and engaging for audiences interested in AI and content marketing:

1. **"Harnessing AI for Content Marketing: Real-World Success Stories and Strategies from Top Creators"**
   - Explore how successful marketers and content creators are integrating AI tools into their strategies, highlighting specific case studies and examples that demonstrate real ROI.

2. **"The Future of Content Marketing: Top 10 AI Tools You Need to Know in 2025"**
   - A forward-looking analysis of emerging AI tools that are set to revolutionize content marketing, including predictions on their potential impact on creativity and efficiency.

3. **"From Ideas to Execution: How AI Tools Are Shaping Marketing Campaigns in 2024"**
   - Discuss the latest AI tools available for marketers, focusing on how they can streamline the content creation process, improve audience targeting, and enhance campaign performance.

In [6]:
# Example usage
sample_keywords = ["AI in marketing", "SEO automation", "generative content"]
print(generate_topics(sample_keywords))

Here are five article topics that incorporate the keywords "AI in marketing," "SEO automation," and "generative content," while ensuring they are fresh, relevant, and engaging:

1. **"Harnessing AI in Marketing: The Future of Personalized Customer Experiences"**  
   Explore how AI technologies can be utilized to tailor marketing strategies to individual customer preferences, enhancing engagement and conversion rates. Discuss case studies of brands successfully implementing AI-driven personalization.

2. **"SEO Automation: Streamlining Your Strategy with AI Tools"**  
   Delve into the latest SEO automation tools powered by AI, examining how they can improve keyword research, content optimization, and performance tracking. Provide a step-by-step guide on integrating these tools into existing marketing workflows for maximum efficiency.

3. **"The Rise of Generative Content: How AI is Shaping Creative Marketing Strategies"**  
   Investigate the impact of generative content on marketing 